<a href="https://colab.research.google.com/github/dtran421/machine-learning-engineering-demo/blob/main/Machine_Learning_Engineering_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import re
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer